In [1]:
import nltk
import time
import re
import os
import sys
import time
import glob
import numpy as np
# import settings

In [2]:
#h5 = settings.apply_to_all_files()

In [3]:
# ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath(__file__)), ".."))
# DATA_DIR = os.path.join(ROOT_DIR, 'data')
# ADD_DATA_DIR = os.path.join(ROOT_DIR, 'AdditionalFiles')
# SUBSET_FILE = os.path.join(ADD_DATA_DIR, 'subset_msd_summary_file.h5')
# MSD_CODE_PATH = os.path.join(ROOT_DIR, 'MSongsDB')
# sys.path.append(os.path.join(MSD_CODE_PATH, 'PythonSrc'))

In [4]:
# import hdf5_getters as GETTERS

In [5]:
def posTagSentence(inStr):
    text = nltk.word_tokenize(inStr.lower())
    return nltk.pos_tag(text)

In [6]:
#Use this one to keep strings lowercase
def nerTagSentence(tagList):
    return (nltk.ne_chunk(tagList, binary=True))

In [7]:
def findSongToArtistQuery(taggedStr):
    #Regex for finding artist of a song
    exStA = re.compile('((who.*)?((sang|made|sings)|(is.*(artist|singer|band).*(of|from|for))|\
                     ((what|which).*)?(artist|SINGER|band) ?(sang|made|of)?))')
    #Regex for finding songs of an artist
    exAtS = re.compile('(((what.*)|(which.*))?((song)s?.*?of.*?)|((song)s?.*?(did)))')
    foundStA = re.search(exStA, taggedStr)
    foundAtS = re.search(exAtS, taggedStr)
    if foundStA is not None:
        print("QUERY StA: ", re.sub(r'([^\w\s]|[A-Z])', '',taggedStr[foundStA.span()[1]:]).lstrip().rstrip().title())
        return(re.sub(r'([^\w\s]|[A-Z])', '',taggedStr[foundStA.span()[1]:]).lstrip().rstrip())
    elif foundAtS is not None:
        print("QUERY AtS: ", (re.sub(r'((did|write|sing|make)|[^\w\s]|[A-Z])', '',taggedStr[foundAtS.span()[1]:]).lstrip().rstrip()))
        return(re.sub(r'((did|write|sing|make)|[^\w\s]|[A-Z])', '',taggedStr[foundAtS.span()[1]:]).lstrip().rstrip().title())
    else:
        print("Query not understood")
        return False
    

In [8]:
text = "What songs did Queen make?"
posTagged = posTagSentence(text)
nerTagged = nerTagSentence(posTagged)
parse_string = ' '.join(str(nerTagged).split())
print("Found query: ", findSongToArtistQuery(parse_string))

QUERY AtS:  queen
Found query:  Queen


In [9]:
#Hi and Hello are currently recognised as NNP's
def initialiseChat():
    print("Hi, I am Name. You can ask anything that has to do with music!")
    time.sleep(.3)
    inStr = posTagSentence(str(input()))
    name = ""
    for el in range (0, len(inStr)):
        if inStr[el][1] == 'NNP':
            name = " " + inStr[el][0]
    print ("Hello" + name + ", What would you like to know?")
    continueChat()

In [10]:
def continueChat():
    while True:
        inStr = str(input())
        if exitCheck(inStr.lower()):
            print("Very well, I hope I could be of help.")
            break
        elif inStr.lower() == "yes":
            print("What would you like to know?")
        else:
            posTagged = posTagSentence(inStr)
            nerTagged = nerTagSentence(posTagged)
            parse_string = ' '.join(str(nerTagged).split())
            findSongToArtistQuery(parse_string)
            print("Is there anything else I can help you with?")

In [11]:
#Assumes inStr is already POS-tagged
#Returns the predicate of what the question is about
def detectQuestions(tagList):
    start = 0
    end = len(tagList)
    for el in range(0, len(tagList)):
        if tagList[el][1] == 'WP':
            start = el + 1
        if tagList[el][1] == '.' and tagList[el][0] == '?':
            end = el
    pred = tagList[start:end]
    return pred

In [12]:
def exitCheck(inStr):
    negatives = ["no ", "nope ", "n", "no thanks", "bye"]
    for neg in negatives:
        if inStr.startswith(neg):
            return True
    return False

In [13]:
if __name__ == "__main__":
    initialiseChat()

Hi, I am Name. You can ask anything that has to do with music!
yeeeee
Hello, What would you like to know?
nothing
Input:  nothing
Very well, I hope I could be of help.
